# Preprocesamiento de Datos

## Índice de Contenido

[1. Carga de datos brutos](#1.-Carga-de-datos-brutos)  
[2. Transformación de códigos a etiquetas descriptivas](#2.-Transformación-de-códigos-a-etiquetas-descriptivas)  
[3. Extracción de la variable de género](#3.-Extracción-de-la-variable-de-género)  
[4. Creación de etiquetas categóricas para la variable objetivo](#4.-Creación-de-etiquetas-categóricas-para-la-variable-objetivo)  
[5. Verificación de consistencia de los datos](#5.-Verificación-de-consistencia-de-los-datos)  
[6. Guardado de los datos procesados](#6.-Guardado-de-los-datos-procesados)  
[7. Resumen](#7.-Resumen)  

## 1. Carga de datos brutos

Comenzamos cargando el conjunto de datos original 'german.data', que se encuentra en formato de texto 
con valores codificados (tanto numéricos como alfanuméricos). Estos datos requieren procesamiento 
para ser interpretables y utilizables en análisis posteriores.

In [1]:
import pandas as pd
import numpy as np
import os

ruta_archivo = os.path.join('..', 'data', 'raw', 'german.data')

g_credit_data = pd.read_csv(ruta_archivo, sep=' ', header=None, skipinitialspace=True)

print(f"Dimensiones del dataset: {g_credit_data.shape}")
print("\nPrimeras 5 filas de los datos brutos:")
g_credit_data.head()

Dimensiones del dataset: (1000, 21)

Primeras 5 filas de los datos brutos:


,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,A11,6,A34,A43,1169,A65,A75,4,A93,A101,...,A121,67,A143,A152,2,A173,1,A192,A201,1
1,A12,48,A32,A43,5951,A61,A73,2,A92,A101,...,A121,22,A143,A152,1,A173,1,A191,A201,2
2,A14,12,A34,A46,2096,A61,A74,2,A93,A101,...,A121,49,A143,A152,1,A172,2,A191,A201,1
3,A11,42,A32,A42,7882,A61,A74,2,A93,A103,...,A122,45,A143,A153,1,A173,2,A191,A201,1
4,A11,24,A33,A40,4870,A61,A73,3,A93,A101,...,A124,53,A143,A153,2,A173,2,A191,A201,2


Como podemos observar, los datos están en su formato bruto con columnas numeradas y valores codificados 
como "A11", "A32", etc. Estos códigos no son interpretables directamente y necesitan ser transformados 
en etiquetas que describan su significado real según la documentación del conjunto de datos.

El dataset contiene 1000 filas (solicitantes de crédito) y 21 columnas (atributos y la variable objetivo).

## 2. Transformación de códigos a etiquetas descriptivas

Para facilitar el análisis, transformaremos los códigos alfanuméricos a descripciones legibles utilizando
una función auxiliar que mapea estos códigos según la documentación oficial del German Credit Dataset.

In [2]:
from tp_final_fairness_ML.utils.preprocessing import mapear_german_credit_data

g_credit_data = mapear_german_credit_data(g_credit_data)

print("\nPrimeras 5 filas de los datos con etiquetas descriptivas:")
g_credit_data.head()


Primeras 5 filas de los datos con etiquetas descriptivas:


,checking_account,duration,credit_history,purpose,credit_amount,savings_account,employment_since,installment_rate,personal_status_sex,other_debtors,...,property,age,other_installment_plans,housing,number_credits,job,people_liable,telephone,foreign_worker,target
0,< 0 DM,6,critical account / other bank,radio/TV,1169,unknown / no account,>= 7 years,4,"male, single",none,...,real estate,67,none,own,2,skilled / official,1,"yes, registered",yes,1
1,0 <= ... < 200 DM,48,paid duly till now,radio/TV,5951,< 100 DM,1 <= ... < 4 years,2,"female, div/sep/married",none,...,real estate,22,none,own,1,skilled / official,1,none,yes,2
2,no account,12,critical account / other bank,education,2096,< 100 DM,4 <= ... < 7 years,2,"male, single",none,...,real estate,49,none,own,1,unskilled - resident,2,none,yes,1
3,< 0 DM,42,paid duly till now,furniture/equipment,7882,< 100 DM,4 <= ... < 7 years,2,"male, single",guarantor,...,building society / insurance,45,none,for free,1,skilled / official,2,none,yes,1
4,< 0 DM,24,delay in past,car (new),4870,< 100 DM,1 <= ... < 4 years,3,"male, single",none,...,unknown / no property,53,none,for free,2,skilled / official,2,none,yes,2


Ahora los datos son mucho más interpretables. Cada columna tiene un nombre descriptivo 
(como "checking_account", "credit_history", etc.) y los valores son legibles en lenguaje natural 
en lugar de códigos crípticos.

## 3. Extracción de la variable de género

Observamos que la columna 'personal_status_sex' contiene información combinada sobre el estado civil 
y el género del solicitante. Para facilitar nuestro análisis de equidad, extraeremos específicamente 
la información de género en una nueva columna.

In [3]:
status_counts = g_credit_data['personal_status_sex'].value_counts()
print("Distribución de estado civil y género:")
print(status_counts)

Distribución de estado civil y género:
personal_status_sex
male, single                548
female, div/sep/married     310
male, married/widowed        92
male, divorced/separated     50
Name: count, dtype: int64


Podemos observar que la columna 'personal_status_sex' contiene cuatro categorías, donde solo una corresponde 
a mujeres (female, div/sep/married) y las otras tres a hombres con diferentes estados civiles. 

Para facilitar el análisis de equidad por género, necesitamos crear una nueva columna 'genre' que 
contenga exclusivamente la información de género (masculino o femenino).

In [4]:
from tp_final_fairness_ML.utils.preprocessing import extraer_genero

g_credit_data = extraer_genero(g_credit_data, 'genre')

print("\nComparación entre personal_status_sex y genre:")
g_credit_data[['personal_status_sex', 'genre']].head(10)


Comparación entre personal_status_sex y genre:


,personal_status_sex,genre
0,"male, single",male
1,"female, div/sep/married",female
2,"male, single",male
3,"male, single",male
4,"male, single",male
5,"male, single",male
6,"male, single",male
7,"male, single",male
8,"male, divorced/separated",male
9,"male, married/widowed",male


## 4. Creación de etiquetas categóricas para la variable objetivo

La variable objetivo 'target' actualmente está codificada como 1 (bajo riesgo) y 2 (alto riesgo).
Para mejorar la interpretabilidad, especialmente en visualizaciones futuras, crearemos una nueva
columna 'target_label' con etiquetas descriptivas.

In [5]:
g_credit_data['target_label'] = g_credit_data['target'].map({1: 'low_risk', 2: 'high_risk'})

print("\nVerificación de la transformación de target a target_label:")
g_credit_data[['target', 'target_label']].head(10)


Verificación de la transformación de target a target_label:


,target,target_label
0,1,low_risk
1,2,high_risk
2,1,low_risk
3,1,low_risk
4,2,high_risk
5,1,low_risk
6,1,low_risk
7,1,low_risk
8,1,low_risk
9,2,high_risk


## 5. Verificación de consistencia de los datos

Antes de guardar los datos procesados, es fundamental verificar la calidad y consistencia del dataset:
- Comprobar que no hay valores faltantes
- Verificar que los tipos de datos son correctos
- Confirmar que todas las columnas esperadas están presentes

In [6]:
print("\nEstructura del dataset procesado:")
g_credit_data.info()

print("\nEstadísticas descriptivas de las variables numéricas:")
g_credit_data.describe()


Estructura del dataset procesado:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   checking_account         1000 non-null   object
 1   duration                 1000 non-null   int64 
 2   credit_history           1000 non-null   object
 3   purpose                  1000 non-null   object
 4   credit_amount            1000 non-null   int64 
 5   savings_account          1000 non-null   object
 6   employment_since         1000 non-null   object
 7   installment_rate         1000 non-null   int64 
 8   personal_status_sex      1000 non-null   object
 9   other_debtors            1000 non-null   object
 10  residence_since          1000 non-null   int64 
 11  property                 1000 non-null   object
 12  age                      1000 non-null   int64 
 13  other_installment_plans  1000 non-null   object
 14  housin

,duration,credit_amount,installment_rate,residence_since,age,number_credits,people_liable,target
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,20.903000,3271.258000,2.973000,2.845000,35.546000,1.407000,1.155000,1.300000
std,12.058814,2822.736876,1.118715,1.103718,11.375469,0.577654,0.362086,0.458487
min,4.000000,250.000000,1.000000,1.000000,19.000000,1.000000,1.000000,1.000000
25%,12.000000,1365.500000,2.000000,2.000000,27.000000,1.000000,1.000000,1.000000
50%,18.000000,2319.500000,3.000000,3.000000,33.000000,1.000000,1.000000,1.000000
75%,24.000000,3972.250000,4.000000,4.000000,42.000000,2.000000,1.000000,2.000000
max,72.000000,18424.000000,4.000000,4.000000,75.000000,4.000000,2.000000,2.000000


Confirmamos que:
- El dataset tiene 1000 registros y 23 columnas (las 21 originales más 'genre' y 'target_label')
- No hay valores nulos en ninguna columna
- Las variables numéricas y categóricas tienen los tipos de datos adecuados
- Las distribuciones de las variables parecen razonables

## 6. Guardado de los datos procesados

Finalmente, guardamos el dataset procesado en formato CSV para su uso en las etapas posteriores
del proyecto.

In [7]:
ruta_guardado = os.path.join('..', 'data', 'processed', 'german_credit_data.csv')

g_credit_data.to_csv(ruta_guardado, index=False)
print(f"\nDatos procesados guardados en: {ruta_guardado}")


Datos procesados guardados en: ..\data\processed\german_credit_data.csv


## 7. Resumen 

En este notebook hemos completado las siguientes tareas de preprocesamiento:

1. **Carga de datos**: Importamos el conjunto de datos German Credit en su formato bruto
2. **Transformación**: Convertimos los códigos a etiquetas descriptivas en lenguaje natural
3. **Extracción de género**: Aislamos la variable de género para facilitar el análisis de equidad
4. **Etiquetado de target**: Creamos etiquetas descriptivas para la variable objetivo
5. **Verificación**: Comprobamos la consistencia y calidad de los datos
6. **Guardado**: Almacenamos el dataset procesado para su uso posterior